# Classifying Document

In [5]:
import warnings
from tqdm import tqdm
import numpy as np
import pandas as pd
from itertools import accumulate
from IPython.display import Markdown as md
import plotly.graph_objs as go
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torchtext.datasets import AG_NEWS
from torch.utils.data.dataset import random_split
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset

def warn(*args, **kwargs):
  pass

warnings.warn = warn
warnings.filterwarnings('ignore')

## Helper functions

In [6]:
def plot(COST, ACC):
  fig, ax1 = plt.subplots()
  color = 'tab:red'
  ax1.plot(COST, color=color)
  ax1.set_xlabel('epoch', color=color)
  ax1.set_ylabel('total loss', color=color)
  ax1.tick_params(axis='y', color=color)

  ax2 = ax1.twinx()
  color = 'tab:blue'
  ax2.set_ylabel('accuracy', color=color)
  ax2.plot(ACC, color=color)
  ax2.tick_params(axis='y', color=color)
  fig.tight_layout()

  plt.show()

## Import bank dataset

In [13]:
train_iter = iter(AG_NEWS(split='train'))

y, text = next((train_iter))
print(f'Text: {text}')

ag_news_label = {1: 'World', 2: 'Sports', 3: 'Business', 4: 'Sci/Tec'}
print(f'Label: {y}, {ag_news_label[y]}')

num_class = len(set([label for (label, text) in train_iter]))
print(num_class)

Text: Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
Label: 3, Business
4
